Explore ways to extract table info from structured (i.e. non-flat) Excel spreadsheets

In [1]:
import pandas as pd

from pathlib import Path

I'm using the example `2017BhavnaNew` file from the `Deduce-shared` GDrive folder: https://drive.google.com/open?id=1-ppRIL9GdHGBlQugybhnz1irDDEsmVCg

In [2]:
PATH = Path('/data/dl/web/test-tt-pore-water-data.xlsm')

In [3]:
pd.read_excel?

Signature:
pd.read_excel(
    io,
    sheet_name=0,
    header=0,
    names=None,
    index_col=None,
    parse_cols=None,
    usecols=None,
    squeeze=False,
    dtype=None,
    engine=None,
    converters=None,
    true_values=None,
    false_values=None,
    skiprows=None,
    nrows=None,
    na_values=None,
    keep_default_na=True,
    verbose=False,
    parse_dates=False,
    date_parser=None,
    thousands=None,
    comment=None,
    skip_footer=0,
    skipfooter=0,
    convert_float=True,
    mangle_dupe_cols=True,
    **kwds,
)
Docstring:
Read an Excel file into a pandas DataFrame.

Support both `xls` and `xlsx` file extensions from a local filesystem or URL.
Support an option to read a single sheet or a list of sheets.

Parameters
----------
io : str, file descriptor, pathlib.Path, ExcelFile or xlrd.Book
    The string could be a URL. Valid URL schemes include http, ftp, s3,
    gcs, and file. For file URLs, a host is expected. For instance, a local
    file could be /path/t

In [4]:
# I would like to leave the dates untouched at this point,
# but I suspect excel might be having some per-cell datetime set,
# and this is honoring it
DF = (pd.read_excel(PATH, sheet_name=0, header=None)
      # set index to be 1-based to match the source columns
      .shift()[1:]
     )
DF

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
1,TT01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-04-26 00:00:00,depth,volume,NaN,Li,Be,B,Na,Mg,Al,...,HCO3-,NO3(WD),S(WD),Cl(WD),NaN,DOC,depth,NO3(Rifle)/ NO3(LBL),SO4(Rifle)/ SO4(LBL),Cl(Rifle)/ Cl(LBL)
5,NaN,"z, m",mL,NaN,uM,µM,uM,mM,mM,uM,...,mM,mM,mM,mM,NaN,mM,"z, m",NaN,NaN,NaN
6,NaN,-0.5,185,NaN,17.1506,NaN,39.0816,3.44001,2.10883,0.00324339,...,2.13567,NaN,4.29395,2.35942,NaN,1.26,-0.5,NaN,0.909419,0.836179
7,NaN,-0.95,192,NaN,11.5239,NaN,67.4826,21.1641,10.494,0.00346711,...,2.57567,NaN,24.6873,7.36136,NaN,1.65,-0.95,NaN,0.953187,1.04534
8,NaN,-1.5,80,NaN,20.2655,NaN,41.6368,88.4052,28.0345,0.00263293,...,4.67,NaN,63.9071,38.5618,NaN,3.77,-1.5,NaN,0.905735,0.870137
9,NaN,-1.94,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.145,NaN,13.2094,7.59916,NaN,NaN,-1.94,NaN,0.932133,0.887743
10,NaN,-2.43,43,NaN,14.3186,NaN,73.9796,36.1096,13.5898,0.00614303,...,4.19667,NaN,35.4888,11.8344,NaN,1.62,-2.43,NaN,0.916393,0.91165


In [5]:
def is_header_row(row):
    # this should obviously be customized/adapted
    return 'depth' in row.tolist()

In [6]:
def to_intervals(seq):
    return [(seq[i], seq[i + 1] -1) for i in range(len(seq) - 1)]

def to_slices(endpoints):
    return [slice(start, end) for start, end in endpoints]

In [7]:
header_indices = [2, 16, 30, 44, 58, 72, 86, 100, 114, 128, 142, 156, 170, 184, 198, 212, 226, 240, 255, 269]
to_slices(to_intervals(header_indices))

[slice(2, 15, None),
 slice(16, 29, None),
 slice(30, 43, None),
 slice(44, 57, None),
 slice(58, 71, None),
 slice(72, 85, None),
 slice(86, 99, None),
 slice(100, 113, None),
 slice(114, 127, None),
 slice(128, 141, None),
 slice(142, 155, None),
 slice(156, 169, None),
 slice(170, 183, None),
 slice(184, 197, None),
 slice(198, 211, None),
 slice(212, 225, None),
 slice(226, 239, None),
 slice(240, 254, None),
 slice(255, 268, None)]

In [8]:
import re

def get_datetimes(str_with_dates, formats=None, pattern=None, to_datetime=None, **kwargs):
    # not used ATM
    # formats = formats or ['%m/%d/%Y']
    pattern = pattern or r'\d{1,2}.\d{1,2}.\d{2,4}'
    to_datetime = to_datetime or pd.to_datetime
    
    matches = re.findall(pattern, str_with_dates)
    return [to_datetime(match, **kwargs) for match in matches]

get_datetimes('05/13/2019, and 05/13/2009 and some other day')

[Timestamp('2019-05-13 00:00:00'), Timestamp('2009-05-13 00:00:00')]

In [9]:
def process_single(raw):
    headers = raw.iloc[0, 1:].tolist()
    headers = ['_cmt_port'] + headers
    # we don't care about units for the time being
    # units = ...
    
    # TODO improve date/timestamp processing
    # in general I would try to extract an indicative date
    # to be upcasted to a datetime type,
    # and leave the raw string in a separate column
    # maybe use a consistent prefix to refer to these "as is from the source file" columns?
    date_raw = raw.iloc[0, 0]

    # this could be either a timestamp-like (set in the orig spreadsheet), or a string
    if isinstance(date_raw, str):
            # as a basic criterion, just take the first date mentioned
            date_extracted = get_datetimes(date_raw)[0]
    else:
        date_extracted = date_raw

    # print(date_raw, date_extracted)
    
    data = raw.iloc[2:, :]
    
    # must use data.values as the main ctor arg
    df = (pd.DataFrame(data.values, columns=headers)
          # TODO verify if this is robust wrt changes in the previous processing steps
          .assign(_source_row=data.index)
          # important to use a function here, so that "d.columns" refers to the current dataframe
          .pipe(lambda d: d.loc[:, d.columns.dropna()])
          .pipe(lambda d: d.loc[:, ~d.columns.duplicated()])
          .assign(_date=date_extracted, _source_date=date_raw)
          # TODO set index here or later?
         )
    print(df.shape)

    return df

In [12]:
def split_to_tables(df):
    header_rows = df[lambda d: d.apply(is_header_row, axis=1)]
    split_idx = header_rows.index.tolist()
    print(split_idx)

    separate = [df.loc[slice_]
                for slice_ in to_slices(to_intervals(split_idx))
               ]
    
    return separate
    
def process(raw):
    
    return pd.concat([process_single(single_df)
                      for single_df in split_to_tables(raw)],
                      sort=False,
                      axis=0,
                     )

In [13]:
(DF
 .dropna(axis=0, how='all')
 .dropna(axis=1, how='all')
#  [lambda d: d.apply(is_header_row, axis=1)]
#  .pipe(split_to_tables)[0]
#  .pipe(process_single2)
 .pipe(process)
 .set_index(['_date', '_source_row'])
#  .index.tolist()
)

[4, 18, 32, 46, 60, 74, 88, 102, 116, 130, 144, 158, 172, 186, 200, 214, 228, 242, 257, 271]
(10, 64)
(10, 64)
(10, 64)
(10, 62)
(10, 61)
(10, 62)
(10, 62)
(10, 62)
(10, 62)
(10, 62)
(10, 62)
(10, 62)
(10, 62)
(10, 62)
(10, 62)
(10, 62)
(10, 62)
(10, 62)
(10, 61)


_cmt_port  depth volume       Li           Be  \
_date      _source_row                                                   
2013-04-26 6                   NaN   -0.5    185  17.1506          NaN   
           7                   NaN  -0.95    192  11.5239          NaN   
           8                   NaN   -1.5     80  20.2655          NaN   
           9                   NaN  -1.94     27      NaN          NaN   
           10                  NaN  -2.43     43  14.3186          NaN   
           11                  NaN  -2.94      0      NaN          NaN   
           12                  NaN  -3.36     48  14.8487          NaN   
           13           CMT port 1  -3.97    NaN      NaN          NaN   
           14           CMT port 2  -5.49    NaN      NaN          NaN   
           15           CMT port 3  -7.02    NaN      NaN          NaN   
2013-05-03 20                  NaN   -0.5    165  17.5139          NaN   
           21                  NaN  -0.95     60   12.696          NaN   
           22                  NaN   -1.5     15  33.5759          NaN   
           23                  NaN  -1.94      0      NaN          NaN   
           24                  NaN  -2.43     29  15.9509          NaN   
           25                  NaN  -2.94      0      NaN          NaN   
           26                  NaN  -3.36     26      NaN          NaN   
           27           CMT port 1  -3.97    NaN      NaN          NaN   
           28           CMT port 2  -5.49      y  17.6916          NaN   
           29           CMT port 3 -7.015      y    16.89          NaN   
2013-05-15 34                  NaN   -0.5    132  17.7294          NaN   
           35                  NaN  -0.95     70   16.731          NaN   
           36                  NaN   -1.5     20  26.7848          NaN   
           37                  NaN  -1.94      0      NaN          NaN   
           38                  NaN  -2.43     41  23.1706          NaN   
           39                  NaN  -2.94     14  24.1465          NaN   
           40                  NaN  -3.36      0      NaN          NaN   
           41           CMT port 1  -3.97    NaN      NaN          NaN   
           42           CMT port 2  -5.49      y  15.3277          NaN   
           43           CMT port 3 -7.015      y  17.4247          NaN   
...                            ...    ...    ...      ...          ...   
2014-03-28 230                 NaN   -0.5    NaN      NaN          NaN   
           231                 NaN  -0.95     75   7.9495  -0.00413333   
           232                 NaN   -1.5    105  14.7387   -0.0374667   
           233                 NaN  -1.94     60  30.4908    0.0129333   
           234                 NaN  -2.43    NaN      NaN          NaN   
           235                 NaN  -2.94    NaN      NaN          NaN   
           236                 NaN  -3.36    NaN      NaN          NaN   
           237          CMT port 1  -3.97    NaN      NaN          NaN   
           238          CMT port 2  -5.49      y  14.9878  -0.00208889   
           239          CMT port 3  -7.02      y  17.8107   0.00951111   
2014-04-11 244                 NaN   -0.5     30   10.449  -0.00211111   
           245                 NaN  -0.95     40  14.1795   0.00224444   
           246                 NaN   -1.5     25  21.6767      -0.0446   
           247                 NaN  -1.94     65  29.5999       0.4638   
           248                 NaN  -2.43    NaN      NaN          NaN   
           249                 NaN  -2.94    NaN      NaN          NaN   
           250                 NaN  -3.36    NaN      NaN          NaN   
           251          CMT port 1  -3.97    NaN      NaN          NaN   
           252          CMT port 2  -5.49      y  15.9667       0.0011   
           253          CMT port 3  -7.02      y  18.5513 -6.66667e-05   
2014-04-24 259                 NaN   -0.5    NaN  12.8562 -0.000711111   
           260               